## Imports

In [ ]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus, value #Linear Optimization Library

## Load Given Variables

#### N+1 cities (C0 ... Cn)

In [ ]:
C = ['Allentown', 'Charlotte', 'Chicago', 'Dallas', 'Fontana', 'Nashville', 'Portland', 'Tucson']
C

#### M+1 products (P0 ... Pn)

In [ ]:
production_matrix = pd.read_excel("SBD2.xlsx", sheet_name="ProductionMatrix", header=1)
production_matrix = production_matrix.drop(['(blank)'], axis=1)
production_matrix = production_matrix.iloc[:-2]
production_matrix = production_matrix[["Row Labels", "Grand Total"]]
P = production_matrix["Row Labels"].tolist()
P

#### Contribution Margin for Pm in Cn [Rm,n]

In [ ]:
margin_matrix = pd.read_excel("SBD2.xlsx", sheet_name="MarginMatrix", header=1)
margin_matrix = margin_matrix.drop(['(blank)','Grand Total'], axis=1)
margin_matrix = margin_matrix.iloc[:-2]
margin_matrix = margin_matrix.fillna(0)
R = margin_matrix.drop(columns=["Row Labels"])
R

#### Max Capacity in Cn [T0 to Tn]

In [ ]:
city_cap_matrix = pd.read_excel("SBD2.xlsx", sheet_name="CityCapacity")
city_cap_matrix = city_cap_matrix.rename(columns={"Unnamed: 0": "Capacity Metric"})
city_cap_matrix = city_cap_matrix[city_cap_matrix["Capacity Metric"] == "Max Capacity"]
T = city_cap_matrix.drop(columns=["Capacity Metric"])
T = T.iloc[0].tolist()
T

#### Total Required Production of Pm in all cities [D0, ... Dm]

In [ ]:
D = production_matrix["Grand Total"].tolist()
D

In [ ]:
prod_matrix = pd.read_excel("SBD2.xlsx", sheet_name="ProductionMatrix", header=1)
prod_matrix = prod_matrix.drop(['(blank)'], axis=1)
prod_matrix = prod_matrix.iloc[:-2]
prod_matrix = prod_matrix.drop(columns=["Row Labels", "Grand Total"])
prod_matrix = prod_matrix.fillna(0)

prod_matrix

#### Calculate # of Cities of # Products

In [ ]:
N = len(C) #number of cities
M = len(P) #number of products

## Build the LP Optimization

#### (A) Define Decision Variables


In [ ]:
# Create decision variable matrix: A[m,n] = production allocation in A[product, city]
A = LpVariable.dicts(
    "Production",
    ((m, n) for m in range(0,M) for n in range(0,N)),
    lowBound=0, #non-negativity for dec. vars
    cat='Continuous'  # Use 'Integer' if units must be whole numbers
)

#### (B) Define Objective Function: Maximize Profit (Using Contribution Margin Here)

In [ ]:
# Define Objective Function
model = LpProblem("OptimizeAllocation", LpMaximize) #Define the model

objective_expr = lpSum(A[m, n] * R.iloc[m, n] for m in range(0,M) for n in range(0,N)) #Maximizing CM
model += objective_expr, "Objective"


#### (C) Define Constraints

In [ ]:

#4. For every city, the SUM of the cities produciton <= total city capacity

for n in range(0,N):

    model += lpSum(A[m,n] for m in range(0,M)) <= T[n], f"Constraint1_{n}"


In [ ]:
#2. For every combination Cn, Pm that is incompatible 
#(Respect Product Capability)

for n in range(0,N):
    for m in range(0,M):
        if R.iloc[m,n] == 0:
            model += A[m,n] == 0, f"Constraint2_{m}_and_{n}"

In [ ]:
#3. For every product, SUM of production of product in every city must equal the total requirement of product 

In [ ]:
for m in range(0,M):
    model += lpSum(A[m,n] for n in range(0,N)) == D[m], f"Constraint3_{m}"

In [ ]:
#4. If there is a shutdown city Cs, for every product Px: Ax,s = 0

In [ ]:
S = C.index("Charlotte") #choose shutdown city

for m in range(0,M):
    model += A[m,S] == 0, f"Constraint4_{m}"

In [ ]:
#5. Non-negativity for decision variables (already in dec var creation)

In [ ]:
#6. Add Constraints to ensure some product mix

In [ ]:
for m in range(0,M):
    for n in range(0,N):
        if n == S: #cant force this constraint on a shutdown city
            continue
        model += A[m,n] >= 0.25 * prod_matrix.iloc[m,n], f"Constraint6_1_{m}_and_{n}"
        model += A[m,n] <= 3.5 * prod_matrix.iloc[m,n], f"Constraint6_2_{m}_and_{n}"

In [ ]:
# Solve Model
model.solve()

In [ ]:
print("Status:", LpStatus[model.status])

In [ ]:
import pandas as pd

# Initialize list for (product, city, value)
data = []

for var in model.variables():
    if var.name.startswith("Production_") and var.varValue and var.varValue > 0:
        # Parse index from variable name
        stripped = var.name.replace("Production_(", "").replace(")", "")
        m_str, n_str = stripped.split(",_")
        m = int(m_str)
        n = int(n_str)

        # Map indices to names
        product_name = P[m]
        city_name = C[n]
        value = var.varValue

        data.append((product_name, city_name, value))

# Create long-format DataFrame
df = pd.DataFrame(data, columns=["Product", "City", "Production"])

# Pivot to wide format like the Excel screenshot
matrix_df = df.pivot(index="Product", columns="City", values="Production").fillna(0)

# Optional: Round for display
matrix_df = matrix_df.round(2)

# Show it
matrix_df


In [ ]:
# Export to Excel
matrix_df.to_excel("production_output11.xlsx", sheet_name="Production Plan")
